# Mixing Experiments for Kikuchi Patterns

This notebook explores how to mix two real experimental patterns A and B into a synthetic C. By default it uses BCC/FCC reference patterns from `data/raw/Double Pattern Data/Good Pattern/`.

Run this notebook from the `notebooks/` folder or the repository root. The code will auto-detect the repo root and add it to `sys.path` so `src/` imports work.

Use the interactive controls (if ipywidgets is installed) to sweep x/y weights, choose mixing pipelines, and preview the output.


## Goals

- Mix two pure patterns (A/B) without requiring a target C
- Compare mixing pipelines (normalize-then-mix vs mix-then-normalize)
- Sweep x (A weight) and y (B weight) combinations to see how C changes
- Add blur, noise, and gamma to mimic detector effects
- Build intuition for a sensible mixing strategy on real experimental data


In [ ]:
from __future__ import annotations

import sys
from pathlib import Path

import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

# Repo path setup
ROOT = Path("..").resolve()
if not (ROOT / "src").exists():
    ROOT = Path(".").resolve()
if str(ROOT) not in sys.path:
    sys.path.insert(0, str(ROOT))

# Optional dependencies
try:
    import ipywidgets as widgets
    from IPython.display import display
    HAS_WIDGETS = True
except Exception:
    widgets = None
    display = None
    HAS_WIDGETS = False

try:
    from scipy.ndimage import gaussian_filter
    HAS_SCIPY = True
except Exception:
    gaussian_filter = None
    HAS_SCIPY = False

print(f"ROOT: {ROOT}")
print(f"ipywidgets: {HAS_WIDGETS}, scipy: {HAS_SCIPY}")


In [ ]:
# Default file paths (override as needed)
A_PATH = ROOT / "data" / "raw" / "Double Pattern Data" / "Good Pattern" / "Perfect_BCC-1.bmp"
B_PATH = ROOT / "data" / "raw" / "Double Pattern Data" / "Good Pattern" / "Perfect_FCC-1.bmp"

# Preprocessing settings
USE_MASK = True
DETECT_EXISTING_MASK = True
ZERO_TOLERANCE = 5e-4
OUTSIDE_ZERO_FRACTION = 0.98

NORMALIZE_ENABLED = False
NORMALIZE_METHOD = "min_max"  # min_max, max, percentile, histogram_equalization
SMART_NORMALIZE = True
HIST_BINS = 1024
PERCENTILE = (1.0, 99.0)

# Mixing settings (x=weight for A, y=weight for B)
PIPELINES = ("normalize_then_mix", "mix_then_normalize")
PIPELINE = PIPELINES[0]
WEIGHT_A = 0.6  # x
WEIGHT_B = 0.4  # y
NORMALIZE_WEIGHTS = True  # normalize x/y to sum to 1 before mixing
ALLOW_NON_16BIT = True  # allow non-16-bit inputs for exploration

BLUR_SIGMA = 0.0
GAUSS_NOISE_STD = 0.0
POISSON_LAMBDA = 0.0
GAMMA = 1.0
EXPOSURE_GAIN = 1.0


In [ ]:
from src.preprocessing.mask import apply_circular_mask, build_circular_mask, detect_circular_mask
from src.preprocessing.normalise import normalize_image
from src.utils.io import write_image_16bit


def to_float01_any(image: np.ndarray, path: Path) -> np.ndarray:
    # Convert image to float32 in [0, 1], handling 16-bit and optional non-16-bit inputs.
    if image.ndim == 3:
        image = image[..., 0]
    if image.dtype == np.uint16:
        return image.astype(np.float32) / 65535.0
    if not ALLOW_NON_16BIT:
        raise ValueError(f"Expected 16-bit image at {path}, got dtype={image.dtype}.")
    print(f"Warning: {path} is {image.dtype}; scaling to [0, 1] for exploration.")
    if image.dtype == np.uint8:
        return image.astype(np.float32) / 255.0
    if np.issubdtype(image.dtype, np.integer):
        max_val = np.iinfo(image.dtype).max
        return image.astype(np.float32) / float(max_val)
    if np.issubdtype(image.dtype, np.floating):
        max_val = float(image.max())
        if max_val > 1.0:
            return (image / max_val).astype(np.float32)
        return image.astype(np.float32)
    raise ValueError(f"Unsupported image dtype at {path}: {image.dtype}")


def load_image(path: Path) -> np.ndarray:
    # Load an image and convert to float32 in [0, 1].
    with Image.open(path) as img:
        image = np.array(img)
    return to_float01_any(image, path)


def compute_mask(image: np.ndarray) -> tuple[np.ndarray, dict]:
    # Create a circular mask and optionally detect existing masking
    mask = build_circular_mask(image.shape)
    meta = {}
    if DETECT_EXISTING_MASK:
        detected, fraction = detect_circular_mask(
            image,
            mask,
            zero_tolerance=ZERO_TOLERANCE,
            outside_zero_fraction=OUTSIDE_ZERO_FRACTION,
        )
        meta["detected"] = detected
        meta["outside_zero_fraction"] = fraction
    return mask, meta


def preprocess(image: np.ndarray, mask: np.ndarray | None) -> np.ndarray:
    # Apply optional masking and normalization
    if USE_MASK and mask is not None:
        image = apply_circular_mask(image, mask)
    if NORMALIZE_ENABLED:
        image = normalize_image(
            image,
            method=NORMALIZE_METHOD,
            histogram_bins=HIST_BINS,
            percentile=PERCENTILE,
            mask=mask,
            smart_minmax=SMART_NORMALIZE,
        )
    return image.astype(np.float32)


def apply_gamma(image: np.ndarray, gamma: float) -> np.ndarray:
    # Gamma correction in [0, 1]
    if gamma <= 0:
        raise ValueError("gamma must be > 0")
    return np.clip(image, 0.0, 1.0) ** gamma


def apply_exposure(image: np.ndarray, gain: float) -> np.ndarray:
    # Linear exposure scaling
    return np.clip(image * gain, 0.0, 1.0)


def apply_blur(image: np.ndarray, sigma: float) -> np.ndarray:
    # Gaussian blur if scipy is available
    if sigma <= 0:
        return image
    if not HAS_SCIPY:
        raise RuntimeError("scipy is required for blur. Install scipy or set BLUR_SIGMA=0.")
    return gaussian_filter(image, sigma=sigma).astype(np.float32)


def apply_noise(image: np.ndarray, std: float, poisson_lambda: float, rng: np.random.Generator) -> np.ndarray:
    # Add Gaussian and Poisson noise
    noisy = image
    if std > 0:
        noise = rng.normal(0.0, std, size=image.shape).astype(np.float32)
        noisy = np.clip(noisy + noise, 0.0, 1.0)
    if poisson_lambda > 0:
        scaled = np.clip(noisy * poisson_lambda, 0.0, None)
        noisy = rng.poisson(scaled).astype(np.float32) / float(poisson_lambda)
        noisy = np.clip(noisy, 0.0, 1.0)
    return noisy


def resolve_weights(weight_a: float, weight_b: float, normalize_weights: bool) -> tuple[float, float]:
    # Optionally normalize weights to sum to 1 for mixing.
    if not normalize_weights:
        return weight_a, weight_b
    total = weight_a + weight_b
    if total <= 0:
        raise ValueError("weight_a + weight_b must be > 0 when normalize_weights is True.")
    return weight_a / total, weight_b / total


def describe_mix(weight_a: float, weight_b: float, normalize_weights: bool, pipeline: str) -> str:
    # Format a short description for plots.
    if normalize_weights:
        try:
            w_a, w_b = resolve_weights(weight_a, weight_b, True)
        except ValueError:
            return f"{pipeline} | invalid weights (sum <= 0)"
        return f"{pipeline} | wA={w_a:.3f}, wB={w_b:.3f} (normalized)"
    return f"{pipeline} | x={weight_a:.3f}, y={weight_b:.3f}"


def mix_images(
    image_a: np.ndarray,
    image_b: np.ndarray,
    weight_a: float,
    weight_b: float,
    pipeline: str,
    mask: np.ndarray | None,
    normalize_weights: bool,
) -> np.ndarray:
    # Mix two images using selected pipeline and weights.
    weight_a, weight_b = resolve_weights(weight_a, weight_b, normalize_weights)
    if pipeline == "normalize_then_mix":
        a_norm = normalize_image(
            image_a,
            method=NORMALIZE_METHOD,
            histogram_bins=HIST_BINS,
            percentile=PERCENTILE,
            mask=mask,
            smart_minmax=SMART_NORMALIZE,
        ) if NORMALIZE_ENABLED else image_a
        b_norm = normalize_image(
            image_b,
            method=NORMALIZE_METHOD,
            histogram_bins=HIST_BINS,
            percentile=PERCENTILE,
            mask=mask,
            smart_minmax=SMART_NORMALIZE,
        ) if NORMALIZE_ENABLED else image_b
        mixed = weight_a * a_norm + weight_b * b_norm
    elif pipeline == "mix_then_normalize":
        mixed = weight_a * image_a + weight_b * image_b
        if NORMALIZE_ENABLED:
            mixed = normalize_image(
                mixed,
                method=NORMALIZE_METHOD,
                histogram_bins=HIST_BINS,
                percentile=PERCENTILE,
                mask=mask,
                smart_minmax=SMART_NORMALIZE,
            )
    else:
        raise ValueError(f"Unknown pipeline: {pipeline}")
    if mask is not None and USE_MASK:
        mixed = apply_circular_mask(mixed, mask)
    return np.clip(mixed, 0.0, 1.0).astype(np.float32)


In [ ]:
# Load images
image_a = load_image(A_PATH)
image_b = load_image(B_PATH)

mask, mask_meta = (None, {})
if USE_MASK:
    mask, mask_meta = compute_mask(image_a)
    print("Mask detection:", mask_meta)

image_a = preprocess(image_a, mask)
image_b = preprocess(image_b, mask)

print("A range", image_a.min(), image_a.max())
print("B range", image_b.min(), image_b.max())


In [ ]:
from matplotlib.patches import Circle


def plot_panel(
    a: np.ndarray,
    b: np.ndarray,
    mix: np.ndarray,
    mask: np.ndarray | None,
    title: str | None = None,
) -> None:
    # Plot A, B, and mixed output with annotations
    fig, axes = plt.subplots(1, 3, figsize=(12, 4))
    axes = np.atleast_1d(axes)

    axes[0].imshow(a, cmap="gray")
    axes[0].set_title("A")
    axes[0].axis("off")

    axes[1].imshow(b, cmap="gray")
    axes[1].set_title("B")
    axes[1].axis("off")

    axes[2].imshow(mix, cmap="gray")
    axes[2].set_title("C (synthetic)")
    axes[2].axis("off")

    if mask is not None:
        height, width = a.shape
        center = ((width - 1) / 2.0, (height - 1) / 2.0)
        radius = min(height, width) / 2.0
        for ax in axes:
            ax.add_patch(Circle(center, radius, fill=False, color="yellow", linewidth=1.0))

    if title:
        fig.suptitle(title)

    plt.tight_layout()
    plt.show()


In [ ]:
# Single run with current parameters
rng = np.random.default_rng(0)

title = describe_mix(WEIGHT_A, WEIGHT_B, NORMALIZE_WEIGHTS, PIPELINE)

mixed = mix_images(
    image_a,
    image_b,
    weight_a=WEIGHT_A,
    weight_b=WEIGHT_B,
    pipeline=PIPELINE,
    mask=mask,
    normalize_weights=NORMALIZE_WEIGHTS,
)

mixed = apply_exposure(mixed, EXPOSURE_GAIN)
mixed = apply_gamma(mixed, GAMMA)
mixed = apply_blur(mixed, BLUR_SIGMA)
mixed = apply_noise(mixed, GAUSS_NOISE_STD, POISSON_LAMBDA, rng)

plot_panel(image_a, image_b, mixed, mask, title=title)


In [ ]:
# Weight grid preview
GRID_WEIGHT_VALUES = [0.2, 0.5, 0.8]
APPLY_EFFECTS_IN_GRID = False

rng = np.random.default_rng(1)

for pipeline in PIPELINES:
    n = len(GRID_WEIGHT_VALUES)
    fig, axes = plt.subplots(n, n, figsize=(3 * n, 3 * n))
    axes = np.atleast_2d(axes)
    for i, weight_a in enumerate(GRID_WEIGHT_VALUES):
        for j, weight_b in enumerate(GRID_WEIGHT_VALUES):
            mix = mix_images(
                image_a,
                image_b,
                weight_a,
                weight_b,
                pipeline,
                mask,
                NORMALIZE_WEIGHTS,
            )
            if APPLY_EFFECTS_IN_GRID:
                mix = apply_exposure(mix, EXPOSURE_GAIN)
                mix = apply_gamma(mix, GAMMA)
                mix = apply_blur(mix, BLUR_SIGMA)
                mix = apply_noise(mix, GAUSS_NOISE_STD, POISSON_LAMBDA, rng)

            ax = axes[i, j]
            ax.imshow(mix, cmap="gray")
            if NORMALIZE_WEIGHTS:
                norm_a, norm_b = resolve_weights(weight_a, weight_b, True)
                label = f"x={weight_a:.2f}, y={weight_b:.2f}
wA={norm_a:.2f}, wB={norm_b:.2f}"
            else:
                label = f"x={weight_a:.2f}, y={weight_b:.2f}"
            ax.set_title(label, fontsize=8)
            ax.axis("off")

    fig.suptitle(f"{pipeline} | weight grid")
    plt.tight_layout()
    plt.show()


In [ ]:
# Interactive widgets (if ipywidgets is installed)
if not HAS_WIDGETS:
    print("ipywidgets is not installed. Use the manual parameter cell above.")
else:
    def _run_interactive(weight_a, weight_b, normalize_weights, pipeline, blur_sigma, noise_std, gamma, exposure_gain):
        if normalize_weights and (weight_a + weight_b) <= 0:
            print("weight_a + weight_b must be > 0 when normalize_weights is True.")
            return
        rng = np.random.default_rng(2)
        mix = mix_images(
            image_a,
            image_b,
            weight_a,
            weight_b,
            pipeline,
            mask,
            normalize_weights,
        )
        mix = apply_exposure(mix, exposure_gain)
        mix = apply_gamma(mix, gamma)
        mix = apply_blur(mix, blur_sigma)
        mix = apply_noise(mix, noise_std, 0.0, rng)
        title = describe_mix(weight_a, weight_b, normalize_weights, pipeline)
        plot_panel(image_a, image_b, mix, mask, title=title)

    widgets.interact(
        _run_interactive,
        weight_a=widgets.FloatSlider(
            min=0.0,
            max=1.0,
            step=0.05,
            value=WEIGHT_A,
            description="x (A)",
        ),
        weight_b=widgets.FloatSlider(
            min=0.0,
            max=1.0,
            step=0.05,
            value=WEIGHT_B,
            description="y (B)",
        ),
        normalize_weights=widgets.Checkbox(value=NORMALIZE_WEIGHTS, description="normalize x/y"),
        pipeline=widgets.Dropdown(options=PIPELINES, value=PIPELINE, description="pipeline"),
        blur_sigma=widgets.FloatSlider(min=0.0, max=3.0, step=0.2, value=BLUR_SIGMA),
        noise_std=widgets.FloatSlider(min=0.0, max=0.1, step=0.005, value=GAUSS_NOISE_STD),
        gamma=widgets.FloatSlider(min=0.5, max=2.0, step=0.05, value=GAMMA),
        exposure_gain=widgets.FloatSlider(min=0.5, max=2.0, step=0.05, value=EXPOSURE_GAIN),
    )


In [ ]:
# Save a synthetic C image (optional)
SAVE_OUTPUT = False
OUTPUT_DIR = ROOT / "data" / "synthetic" / "experiments"
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

if SAVE_OUTPUT:
    out_path = OUTPUT_DIR / "synthetic_C_from_A_B.png"
    write_image_16bit(out_path, mixed)
    print(f"Saved {out_path}")
else:
    print("Set SAVE_OUTPUT = True to write synthetic output.")


## Notes

- Keep images in float32 [0, 1] while experimenting. Only save to 16-bit when you decide to export.
- The default A/B patterns are real experimental BCC/FCC references from `data/raw/Double Pattern Data/Good Pattern/`.
- Set ALLOW_NON_16BIT = False to enforce strict 16-bit inputs.
- Use NORMALIZE_WEIGHTS if you want x/y to behave like mixing fractions that sum to 1.
- For more realism, consider adding point spread blur and detector noise.
